### Libraries

In [ ]:
import time
import joblib
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from scipy.stats import ks_2samp
from IPython.display import Image

import shap
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RepeatedStratifiedKFold, LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.tree import export_graphviz
from yellowbrick.model_selection import FeatureImportances

from imblearn.over_sampling import SMOTE, ADASYN

import xgboost as xgb
from plot_learning import *
from sklearn.svm import SVC
from sklearn_rvm import EMRVC
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import GridSearchCV, learning_curve, RandomizedSearchCV
from sklearn.metrics  import average_precision_score, make_scorer, roc_curve,f1_score, precision_score, recall_score, fbeta_score, auc, roc_auc_score, accuracy_score, confusion_matrix, classification_report,precision_recall_curve
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

warnings.filterwarnings('ignore')

### Read 

In [ ]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

X_train = pd.read_csv("X_train.csv", sep = ";")
X_test = pd.read_csv("X_test.csv", sep = ";")

y_train = pd.read_csv("y_train.csv", sep = ";")
y_test = pd.read_csv("y_test.csv", sep = ";")

X_train.head()

In [ ]:
## removendo variáveis 
X_train = X_train.drop([
'H1RFV', 'H1LFV', 'H1RRO'
], axis=1)

X_test = X_test.drop([
'H1RFV', 'H1LFV', 'H1RRO'
], axis=1)

In [ ]:
X_train.columns

In [ ]:
y_train.GR.unique()

In [ ]:
y_train.GR.value_counts()

## Dataprep

In [ ]:
def prepare_targets(y_train, y_test):
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_test_enc = le.transform(y_test)
    
    return y_train_enc, y_test_enc

In [ ]:
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)

In [ ]:
print(Counter(y_train_enc).keys())
print(Counter(y_train_enc).values())

In [ ]:
print(Counter(y_test_enc).keys())
print(Counter(y_test_enc).values())

In [ ]:
file_minmax = "./models/minmax.pickle"
file_smote = "./models/smote.pickle"

minmax = joblib.load(file_minmax)
smote= joblib.load(file_smote)

In [ ]:
X_train_minmax = minmax.transform(X_train)
X_test = minmax.transform(X_test)

In [ ]:
X_resampled, y_resampled = smote.fit_resample(X_train_minmax, y_train_enc)

In [ ]:
print(Counter(y_resampled).keys())
print(Counter(y_resampled).values())

## Predict

In [ ]:
def fit_and_print(model, X_test, y_test):
    y_pred = model.predict(X_test)
    # print("Confusion Matrix: \n", confusion_matrix(y_test_test, y_pred))
    cm = confusion_matrix(y_test, y_pred)
    index =["A", "B", "C"]
    columns =["A", "B", "C"]
    cm_df = pd.DataFrame(cm,columns,index)                      
    plt.figure(figsize=(8,6))  
    sns.heatmap(cm_df, annot=True)
    print("Classification Report: \n", classification_report(y_test, y_pred))  

### Models

In [ ]:
filename_1 = './models/RF_best_bayes_resampled.sav'
filename_2 = './models/SVM_best_bayes_resampled.sav'
filename_3 = './models/LR_best_bayes_resampled.sav'
filename_4 = './models/MLP_best_resampled.sav'
filename_5 = './models/KNN_best_bayes_resampled.sav'
filename_6 = './models/GP_best_bayes_resampled.sav'
filename_7 = './models/GBM_best_bayes_resampled.sav'
filename_8 = './models/LGBM_best_bayes_resampled.sav'
filename_9 = './models/XGB_best_bayes_resampled.sav'


RF_best = joblib.load(filename_1)
SVM_best = joblib.load(filename_2)
LR_best = joblib.load(filename_3)
MLP_best = joblib.load(filename_4)
KNN_best = joblib.load(filename_5)
GP_best = joblib.load(filename_6)
GBM_best = joblib.load(filename_7)
LGBM_best = joblib.load(filename_8)
# XGB_best = joblib.load(filename_9)

### Voting Classifier

In [ ]:
model_RF = pd.Series(RF_best.predict(X_test), name="RF")
model_GP = pd.Series(GP_best.predict(X_test), name="GPC")
model_MLP = pd.Series(MLP_best.predict(X_test), name="MLP")
model_LR = pd.Series(LR_best.predict(X_test), name= "LR")
model_SVM = pd.Series(SVM_best.predict(X_test), name="SVM")
model_GBM = pd.Series(GBM_best.predict(X_test), name="GBM")
model_LGBM = pd.Series(LGBM_best.predict(X_test), name="LGBM")
# model_XGB = pd.Series(XGB_best.predict(X_test), name="XGB")
model_KNN = pd.Series(KNN_best.predict(X_test), name="KNN")

# Concatenate all classifier results
m= pd.concat([model_RF, model_GP, model_MLP, model_LR, model_SVM, model_LGBM, model_GBM, model_KNN],axis=1)

corrmat = np.triu(m.corr(method='kendall'))

g= sns.heatmap(m.corr(method='kendall'),cmap="coolwarm",annot=True, fmt=".2f", mask = corrmat)

In [ ]:
import scipy.stats as stats

tau, p_value = stats.kendalltau(model_RF, model_GP)
p_value

In [ ]:
def fit_and_print_v2(model, X_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test_enc, y_pred)
    index =["A", "B", "C"]
    columns =["A", "B", "C"]
    cm_df = pd.DataFrame(cm,columns,index)                      
    plt.figure(figsize=(8,6))  
    sns.heatmap(cm_df, annot=True)
    print("Classification Report: \n", classification_report(y_test_enc, y_pred))  


In [ ]:
fit_and_print_v2(RF_best, X_test)

In [ ]:
fit_and_print_v2(GP_best, X_test)